# SO8T群Transformer 8bit量子化GGUF変換 (GoogleColab)

このノートブックでは、SO8T群Transformerモデルを8bit量子化GGUF形式に変換します。

## 特徴
- 🚀 **SO8T群構造の保持**: SO(8)群回転と非可換ゲート
- 🧠 **Triality reasoning**: 3つの推論ヘッド（task, safety, authority）
- ⚡ **8bit量子化**: メモリ使用量を大幅削減
- 📦 **GGUF形式**: 効率的なモデル保存と読み込み
- 🔧 **GoogleColab最適化**: メモリ効率と実行速度を最適化

## SO8T群とは？
SO8T群は8次元回転群SO(8)をベースとした特殊なTransformerアーキテクチャです：
- **非可換ゲート**: R_safe → R_cmd の非可換積
- **PET正則化**: 時系列一貫性による群の慣性
- **安全人格**: 学習中に群構造が崩壊しない設計


## 1. 環境セットアップ


In [ ]:
# 必要なライブラリをインストール
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
!pip install transformers accelerate bitsandbytes
!pip install numpy tqdm

# メモリ効率化のための環境変数設定
import os
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True'
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

print("✅ 依存関係のインストール完了")


In [ ]:
import torch
import numpy as np
import json
import time
import gc
from pathlib import Path
from tqdm import tqdm
import logging

# ログ設定
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

# GPU情報表示
if torch.cuda.is_available():
    print(f"🚀 GPU: {torch.cuda.get_device_name(0)}")
    print(f"💾 GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.1f} GB")
else:
    print("⚠️ GPU not available, using CPU")

print("✅ ライブラリインポート完了")
